In [1]:
from pprint import pprint

In [2]:
import torch as th

In [2]:
th.ones(2, 3)


 1  1  1
 1  1  1
[torch.FloatTensor of size 2x3]

In [3]:
a = th.Tensor([[1, 2],[3, 4]])
a


 1  2
 3  4
[torch.FloatTensor of size 2x2]

In [4]:
a_gpu = a.cuda()
print(a_gpu)
a_cpu = a_gpu.cpu()
print(a_cpu)


 1  2
 3  4
[torch.cuda.FloatTensor of size 2x2 (GPU 0)]


 1  2
 3  4
[torch.FloatTensor of size 2x2]



In [5]:
a.view(1, -1)


 1  2  3  4
[torch.FloatTensor of size 1x4]

In [6]:
print(a.mean(dim=0))
print(a.std(dim=1))


 2
 3
[torch.FloatTensor of size 2]


 0.7071
 0.7071
[torch.FloatTensor of size 2]



In [7]:
from torch import autograd as tha
tha.Variable

torch.autograd.variable.Variable

In [8]:
a = tha.Variable(th.ones(2, 3), requires_grad=True)
print(a.data)
print(a.grad)



 1  1  1
 1  1  1
[torch.FloatTensor of size 2x3]

None


In [9]:
x = tha.Variable(th.ones(2), requires_grad=True)
y = 5 * (x + 2) ** 2

o = (1 / 2) * th.sum(y)
o.backward()

x.grad

Variable containing:
 15
 15
[torch.FloatTensor of size 2]

In [8]:
import collections
import gym
import pandas as pd
import roboschool

pd.set_option('display.expand_frame_repr', False)

env_spec_list = gym.envs.registry.all()
envs = collections.defaultdict(list)
envs_df = pd.DataFrame(columns=['name', 'obs_space', 'ac_space', 'obs_range', 'ac_range'])
for i, env_spec in enumerate(env_spec_list):
  try:
    env = env_spec.make()
    obs_space = env.observation_space
    ac_space = env.action_space
    obs_range, ac_range = None, None
    if isinstance(obs_space, gym.spaces.Box):
      obs_range = f'{obs_space.low}:{obs_space.high}'
    if isinstance(ac_space, gym.spaces.Box):
      ac_range = f'{ac_space.low}:{ac_space.high}'
    
    cols = [env_spec.id, obs_space, ac_space, obs_range, ac_range]
    cols_str = [str(c) for c in cols]
    cols_str = [s.replace('Discrete', 'D').replace('Box', 'C').replace('Tuple', 'T')
                for s in cols_str]
    envs_df.loc[i] = cols_str
  
  except gym.error.DependencyNotInstalled:
    envs['deps_not_installed'].append(env_spec)
  except ModuleNotFoundError:
    envs['module_not_found'].append(env_spec)

print(envs_df[['name', 'obs_space', 'ac_space']])


                                     name                                obs_space                                    ac_space
0                                 Copy-v0                                     D(6)                         T(D(2), D(2), D(5))
1                           RepeatCopy-v0                                     D(6)                         T(D(2), D(2), D(5))
2                     ReversedAddition-v0                                     D(4)                         T(D(4), D(2), D(3))
3                    ReversedAddition3-v0                                     D(4)                         T(D(4), D(2), D(3))
4                      DuplicatedInput-v0                                     D(6)                         T(D(2), D(2), D(5))
5                              Reverse-v0                                     D(3)                         T(D(2), D(2), D(2))
6                             CartPole-v0                                    C(4,)                             